In [1]:
import geopandas as gpd 
import pandas as pd 

In [2]:
pc_shp = '/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291'

INPUT_GPK = '/Volumes/T9/Data_downloads/Versik_building_data/2024_03_22_updated_data/UKBuildings_Edition_15_new_format_upn.gpkg'

In [6]:
import os
import geopandas as gpd
import pandas as pd
import glob 


def find_shp_files():
    """
    Find all .shp files in the given directory and its subdirectories.
    """
    one = glob.glob('/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/*/*.shp')
    tw = glob.glob('/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/two_letter_pc_code/*.shp')
    return one + tw 

def calculate_areas(shp_files):
    """
    Load shapefiles, calculate the area of the geometries, and return a DataFrame with the results.
    """
    results = []
    for shp_file in shp_files:
        gdf = gpd.read_file(shp_file)
        print(shp_file)
        
        # Ensure the geometries are in the correct projection for area calculation (e.g., UTM)
        if gdf.crs.is_geographic:
            print('convert utm')
            gdf = gdf.to_crs(gdf.estimate_utm_crs())

        gdf['area'] = gdf.geometry.area
        for _, row in gdf.iterrows():
            results.append({
                'postcode': row['POSTCODE'],  # Adjust this if the column name is different
                'file': shp_file,
                'area': row['area']
            })
    
    results_df = pd.DataFrame(results)
    return results_df

# Define the directory containing the shapefiles
shp_directory =  '/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291'

# Find all .shp files in the directory
shp_files = find_shp_files()

# Calculate the areas and save the results
results_df = calculate_areas(shp_files)

# Save the results to a CSV file
results_df.to_csv('shapefile_areas.csv', index=False)

print("Results saved to shapefile_areas.csv")


/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/b/b.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/e/e.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/g/g.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/l/l.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/m/m.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/w/w.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/s/s.shp
/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_post

In [7]:
results_df

,postcode,file,area
0,B1 1BB,/Volumes/T9/Data_downloads/codepoint_polygons_...,12793.612296
1,B1 1BL,/Volumes/T9/Data_downloads/codepoint_polygons_...,2476.338327
2,B1 1BN,/Volumes/T9/Data_downloads/codepoint_polygons_...,2622.313766
3,B1 1DA,/Volumes/T9/Data_downloads/codepoint_polygons_...,2522.988608
4,B1 1DB,/Volumes/T9/Data_downloads/codepoint_polygons_...,7833.212951
...,...,...,...
1732820,VDE01587,/Volumes/T9/Data_downloads/codepoint_polygons_...,100.000000
1732821,VDE01588,/Volumes/T9/Data_downloads/codepoint_polygons_...,100.000000
1732822,VDE01589,/Volumes/T9/Data_downloads/codepoint_polygons_...,100.000000
1732823,VDE01590,/Volumes/T9/Data_downloads/codepoint_polygons_...,100.000000


In [1]:
import os
import geopandas as gpd
import pandas as pd
import glob 

def find_shp_files():
    """
    Find all .shp files in the given directory and its subdirectories.
    """
    one = glob.glob('/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/*/*.shp')
    tw = glob.glob('/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/two_letter_pc_code/*.shp')
    return one + tw 

def extract_lat_lon(shp_files):
    """
    Load shapefiles, extract the centroid latitude and longitude of the geometries, and return a DataFrame with the results.
    """
    results = []
    for shp_file in shp_files:
        gdf = gpd.read_file(shp_file)
        print(shp_file)
        
        # Ensure the geometries are in the correct projection for centroid calculation (WGS 84)
        if not gdf.crs.is_geographic:
            print('Converting to WGS 84')
            gdf = gdf.to_crs(epsg=4326)

        # Calculate the centroids
        gdf['centroid'] = gdf.geometry.centroid

        for _, row in gdf.iterrows():
            centroid = row['centroid']
            results.append({
                'postcode': row['POSTCODE'],  # Adjust this if the column name is different
                'file': shp_file,
                'latitude': centroid.y,
                'longitude': centroid.x
            })
    
    results_df = pd.DataFrame(results)
    return results_df

# Define the directory containing the shapefiles
shp_directory =  '/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291'

# Find all .shp files in the directory
shp_files = find_shp_files()

# Extract the latitude and longitude and save the results
results_df = extract_lat_lon(shp_files)

# Save the results to a CSV file
results_df.to_csv('postcode_lat_lon.csv', index=False)

print("Results saved to postcode_lat_lon.csv")


/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/b/b.shp
Converting to WGS 84


/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_15092/2978550280.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/e/e.shp
Converting to WGS 84


/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_15092/2978550280.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/one_letter_pc_code/g/g.shp
Converting to WGS 84


...

/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/two_letter_pc_code/sl.shp
Converting to WGS 84


/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_15092/2978550280.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


/Volumes/T9/Data_downloads/codepoint_polygons_edina/Download_all_postcodes_2378998/codepoint-poly_5267291/two_letter_pc_code/de.shp
Converting to WGS 84


/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_15092/2978550280.py:29: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.geometry.centroid


Results saved to postcode_lat_lon.csv
